In [6]:
from common import *
from features.selector import Function, Itemgetter, item_check

In [2]:
_TEST_ = False

In [3]:
if __name__=='__main__':
    _TEST_ = True
    interactive(rows=2000, cols=2000)

## .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!. The actual Features subsets definitions!!!! .!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.!.

In [7]:
def time_slice(e):
    start, stop = e.start if e.start is not None else 0, e.stop if e.stop is not None else 0
    assert start >= 0 and stop >= 0
    return max(start, stop), min(start, stop)

def digit_check(x, s):
    '''checks `s in x` but what follows first occurence of `s` isn't a digit char'''
    try:
        i = x.index(s)
        return i+len(s)>=len(x) or x[i+len(s)] in '_ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz{'
    except ValueError:
        return False
    
def i2s(e):
    if isinstance(e, slice):
        lrg, sml = time_slice(e)
    elif isinstance(e, int):
        lrg, sml = e, 0
    else:
        assert False, 'unexpected input type in function `i2s`'
    if sml == 0:
        return str(lrg)
    elif lrg % 5 == sml % 5 == 0:
        return f'({lrg}-{sml})'
    elif lrg - sml == 1:
        return f'{{{sml}}}'
    else:
        return f'{{{sml}/{lrg}}}'

def gen_getitem(*words):
    return lambda f, a: lambda x: any(digit_check(x, word+i2s(e)) for e in item_check(a) for word in words)

def index_check(x, e):
    if isinstance(e, slice):
        fst, snd = e.start, e.stop
        return x[::-1].find(f'{{{fst}/{snd}}}'[::-1]) == 0
    elif e == 0:
        return x[-1] != '}'
    elif isinstance(e, int):
        return x[::-1].find(f'{{{e}}}'[::-1]) == 0
    else:
        assert False, 'wrong type of argument `e` to function index_check'

def gen_getindex():
    return lambda f, a: lambda x: any(index_check(x, e) for e in item_check(a))


def weirdmix(x, e):
    a = '(it,af,it{1},aoo,acc)'
    b = '(it{1},af{1},it{2},aoo{1},acc{1})'
    i = [None,'{0/1}','{1/2}','{0/2}','{0/3}','{1/3}','{2/3}','{0/4}','{1/4}','{2/4}']
    num = e % 10
    s = a if e < 10 else b
    return x == s+i[num]
    


class FFF(metaclass=Function):
    # BIG NOTE item_check isn't necessary for the __getitem__ definitions anymore, it's done automatically by metaclass
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ super basic ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class AssetEnc(metaclass=Function):
        __call__ = lambda f, x: 'assetCode' in x or 'assetName' in x
        Code = lambda f, x: x  == 'assetCodeId'
        
        class RandMap(metaclass=Function):
            __call__ = lambda f, x: 'randMap32' in x
            squared = Itemgetter(lambda a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e**2))[::-1]) == 0 for e in item_check(a)))
            __getitem__ = (lambda f, a: lambda x:
                any(x[::-1].find(('randMap32_' + str(e))[::-1]) == 0 for e in item_check(a)))
        
        WTFF = lambda f, x: 'WTFF' in x
        InUni = lambda f, x: 'inUniCount' in x
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Since and MinMax pools ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    class Since(metaclass=Function):
        __call__ = lambda f, x: 'Since' in x
        __getitem__ = gen_getitem('Since')
        class index(metaclass=Function):
            __call__ = lambda f, x: x[-1] == '}'
            __getitem__ = gen_getindex()
        Max = lambda f, x: 'max' in x or 'Max' in x
        Min = lambda f, x: 'min' in x or 'Min' in x
        
    class FracRec(metaclass=Function):
        __call__ = lambda f, x: 'fracRecordedDays' in x
        __getitem__ = gen_getitem('fracRecordedDaysSince')
        
    class Volatility(metaclass=Function):
        __call__ = lambda f, x: 'olatility' in x
        __getitem__ = gen_getitem('olatility')
    
    class Drawdown(metaclass=Function):
        __call__ = lambda f, x: 'rawdown' in x
        __getitem__ = gen_getitem('rawdown')
    
    class Drawup(metaclass=Function):
        __call__ = lambda f, x: 'rawup' in x
        __getitem__ = gen_getitem('rawup')
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ Stock day by day values ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class index(metaclass=Function):
        __call__ = lambda f, x: x[-1] == '}'
        __getitem__ = gen_getindex()
    
    class FaceValue(metaclass=Function):
        __call__ = lambda f, x: x in ['open', 'close']
        Open = lambda f, x: x == 'open'
        Close = lambda f, x: x == 'close'
        Volume = lambda f, x: x == 'volume'
        
    class VP(metaclass=Function):
        __call__ = lambda f, x: (x[:2] == 'vp' or x[:4] == 'm_vp') and 'dd' not in x
        __getitem__ = gen_getitem('vp')
        
    class Return(metaclass=Function):
        pure = lambda f, x: all(b not in x for b in
                                ['inSince','axSince','rawdown','rawup','ecordedDays','ssetCode','olume'])
        __call__ = lambda f, x: any(ww in x for ww in ['oo','cc','dd','(oo-aoo)','(cc-acc)','(oo-cc)','(aoo-acc)'])
        class oo(metaclass=Function):
            __call__ = lambda f, x: 'oo' in x and not any(s in x for s in ['aoo','-oo','oo-',',oo','oo,'])
            __getitem__ = gen_getitem('oo')
        class aoo(metaclass=Function):
            __call__ = lambda f, x: 'aoo' in x and not any(s in x for s in ['-aoo','aoo-',',aoo','aoo,'])
            __getitem__ = gen_getitem('aoo')
        class doo(metaclass=Function):
            __call__ = lambda f, x: '(oo-aoo)' in x
            __getitem__ = gen_getitem('(oo-aoo)')
            
        class cc(metaclass=Function):
            __call__ = lambda f, x: 'cc' in x and not any(['acc' in x, '-cc' in x, 'cc-' in x, ',cc' in x, 'cc,' in x])
            __getitem__ = gen_getitem('cc')
        class acc(metaclass=Function):
            __call__ = lambda f, x: 'acc' in x and not any(['-acc' in x, 'acc-' in x, ',acc' in x, 'acc,' in x])
            __getitem__ = gen_getitem('acc')
        class dcc(metaclass=Function):
            __call__ = lambda f, x: '(cc-acc)' in x
            __getitem__ = gen_getitem('(cc-acc)')
            
        class doc(metaclass=Function):
            __call__ = lambda f, x: '(oo-cc)' in x
            __getitem__ = gen_getitem('(oo-cc)')
        class daoc(metaclass=Function):
            __call__ = lambda f, x: '(aoo-acc)' in x
            __getitem__ = gen_getitem('(aoo-acc)')
            
        class dd(metaclass=Function):
            __call__ = lambda f, x: 'dd' in x
            __getitem__ = gen_getitem('dd')
        class vp1dd(metaclass=Function):
            __call__ = lambda f, x: 'vp1dd' in x
            __getitem__ = gen_getitem('vp1dd')
        class vp5dd(metaclass=Function):
            __call__ = lambda f, x: 'vp5dd' in x
            __getitem__ = gen_getitem('vp5dd')
        class vp5dd(metaclass=Function):
            __call__ = lambda f, x: 'vp10dd' in x
            __getitem__ = gen_getitem('vp10dd')
            
        class Volatility(metaclass=Function):
            __call__ = lambda f, x: 'olatility' in x
            __getitem__ = gen_getitem('olatility')
        
        __getitem__ = gen_getitem('oo','aoo','(oo-aoo)','vp1dd','vp5dd','volatility')
        class index(metaclass=Function):
            __call__ = lambda f, x: x[-1] == '}'
            __getitem__ = gen_getindex()
        class digit(metaclass=Function):
            __call__ = lambda f, x: x[-1] in '0123456789)'
            
        mix = Itemgetter(lambda a: lambda x: any(weirdmix(x, e) for e in item_check(a)))
        rr = lambda f, x: 'rr' in x
        it = lambda f, x: x[:3] in ['it', 'it{']
        af = lambda f, x: x[:3] in ['af', 'af{']
        TEN = lambda f, x: any(s+'TEN' in x for s in ['oo','cc','aoo','acc'])
        
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ MARKET ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
    class Time(metaclass=Function):
        __call__ = lambda f, x: x in ['dayOfYear', 'dayOfWeek']
        long = lambda f, x: x == 'dayOfYear'
        short = lambda f, x: x == 'dayOfWeek'
        
    class Market(metaclass=Function):
        __call__ = lambda f, x: 'm_' in x
        Return = lambda f, x: 'oo' in x
        VP = lambda f, x: 'vp' in x
        Volatility = lambda f, x: 'olatility' in x
        __getitem__ = gen_getitem('oo','dd','olatility')

        class Weight(metaclass=Function):
            __call__ = lambda f, x: 'wVP' in x
            
    # =>->~>=>->~>=>->~>=>->~>=>->~>=>->~>=>->~>=>->~> PAIRS =>->~>=>->~>=>->~>=>->~>=>->~>=>->~>=>->~>=>->~>=>->~>
    
    class Corr(metaclass=Function):
        __call__ = lambda f, x: '__corr__' == x[:8]
        __getitem__ = gen_getitem('_')
        aoo = lambda f, x: '_aoo_' in x
        aooTEN = lambda f, x: '_aooTEN_' in x
        aooTTY = lambda f, x: '_aooTTY_' in x
        
    class Unic(metaclass=Function):
        __call__ = lambda f, x: '__unic__' == x[:8]
        __getitem__ = gen_getitem('_')
        aoo = lambda f, x: '_aoo_' in x
        aooTEN = lambda f, x: '_aooTEN_' in x
        aooTTY = lambda f, x: '_aooTTY_' in x
        

In [9]:
assert ''.join(list(FFF.stringify())) # super lazy no-error check